## Imports

In [ ]:
import H5toDataframe as readH5
import influxdb_client as influx
from influxdb_client.client.write_api import SYNCHRONOUS
import conversionServices as ConvServ
import datetime as dt

## API configuration

In [ ]:
bucket = "8bfb0bbb9e31d01a" #test5
org = "H493f0c3e5bf84239"
token = "giJmgHGcBaEt8LvH1kZbfpwrNU3_eYDg32MXqbfkZNyuJHxfyOHTFAyYBsOs7M2s-WKRneuGu849FTT3CNdCHg=="
# Store the URL of your InfluxDB instance
url="https://westeurope-1.azure.cloud2.influxdata.com/"

client = influx.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

write_api = client.write_api(write_options=influx.client.write_api.SYNCHRONOUS)

## Read data from HDF5

In [ ]:
df = readH5.convertAllH5Files()

In [ ]:
# Query script
query_api = client.query_api()
query = ('from(bucket:"test5")\
|> range(start: 0)\
|> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")')

result = query_api.query(org=org, query=query)
# results = []
# for table in result:
#     for record in table.records:
#         results.append((record.get_field(), record.get_value()))
        # print(record)

print(result)


In [ ]:
for name, (frame, attrs) in df.items():
    year = ConvServ.getYearFromDf(frame) # extract year from timestamp
    avg_velo = ConvServ.dataframe_column_average(frame, "velocity") # calculate average velocity
    points = []

    for row in frame.iterrows():

        ts = dt.datetime.fromtimestamp(row[1]["timestamp"])

        p = (influx.Point(name)
        .time(ts) # Muss bereits als datetime vorliegen
        .tag("instrument", attrs["instrument"])
        .tag("cofiguration", attrs["configuration"])
        .tag("year", year)
        .tag("avg_velocity", avg_velo) 
        .field("defect_channel", row[1]["defect_channel"])
        .field("distance", row[1]["distance"])
        .field("magnetization", row[1]["magnetization"])
        .field("velocity", row[1]["velocity"])
        .field("wall_thickness", row[1]["wall_thickness"]))
        
        points.append(p)
    
    write_api.write(bucket=bucket, org=org, record=points)